In [1]:
# The following statement imports the NLTK package.
import nltk

# The following statement imports a class called PlaintextCorpusReader.
from nltk.corpus import PlaintextCorpusReader
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk import sent_tokenize
import string
import re 

import tqdm as notebook_tqdm


In [2]:
# instantiate finBERT
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os

# loop through text file in Data
rootdir = 'Data'
filelist = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filename = os.path.join(subdir, file)
        filelist.append(filename)

filelist = filelist[1:6]
# to be changed to the full list
print(filelist)

['Data/BA_transcripts/2016-Jan-28-BA.N-138020839119-Transcript.txt', 'Data/BA_transcripts/2015-Apr-22-BA.N-140063175207-Transcript.txt', 'Data/BA_transcripts/2022-Oct-26-BA.N-140697322944-Transcript.txt', 'Data/BA_transcripts/2010-Jul-28-BA.N-137028827968-Transcript.txt', 'Data/BA_transcripts/2014-Jan-29-BA.N-139067099110-Transcript.txt']


In [4]:
def textPreprocessing(filename):
    # load text
    file = open(filename, 'rt')
    text = file.read()
    file.close()

    # tokenize into sentences
    tokens = sent_tokenize(text)

    # alpha to lowercase
    tokens = [w.lower() for w in tokens]

    # remove punctuation and whitespaces
    remove = string.punctuation
    remove = re.sub(r"[-]+", "", remove)
    pattern = r"[{}]".format(remove + '-')
    clean_list = []

    for sentence in tokens:
        sentence = re.sub(pattern, "", sentence) 
        sentence = sentence.split()
        sentence = ' '.join(sentence)
        clean_list.append(sentence)

    return clean_list

# no neeed to remove stopwords and conduct lemmatization
# may worsen prediction for transformer


In [5]:
# function that takes returns a document level sentiment score

def sentimentAnalysis(filename):
    clean_list = textPreprocessing(filename)
    # pipeline: API that wraps finBERT sentiment analysis 
    nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)
    results = nlp(clean_list)
    
    negative = 0
    positive = 0
    for element in results:
        if element['label'] == "Negative":
            negative += 1
        elif element['label'] == "Positive":
            positive += 1
    total = len(results)

    sentiment_score = (positive - negative)/ total
    return sentiment_score

    # +ve sentiment score: positive earnings call
    # -ve sentiment score: negative earnings call

    # https://www.alpha-sense.com/blog/engineering/sentiment-score/
    # sentiment score: (positive - negative) / total number of statements





In [6]:
sentiment_score_list = []
for filename in filelist:
    sentiment_score = sentimentAnalysis(filename)
    sentiment_score_list.append(sentiment_score)

print(sentiment_score_list)


[0.29038112522686027, 0.1873873873873874, 0.0865561694290976, 0.052884615384615384, 0.25766871165644173]


In [7]:
# # stopword removal
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english'))
# words = [w for w in words if not w in stop_words]
# print(words[:100])

In [8]:
# # lemmatization
# from nltk.stem import WordNetLemmatizer
# lemmatizer = WordNetLemmatizer()
# lemmatized = [lemmatizer.lemmatize(word) for word in words]
# print(lemmatized) 